In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np# data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import collections 
from itertools import chain
import torch 
import time 

from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import CrossEntropyLoss

## 项目folder
from evaluation import islegal, aggregate_f1
from models import BertSpan
from dataset import SpanDataset

from src.dataset.converter import data_loader 
from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.evaluation import span_inference
from src.metric import  seq_span_metrics, tag_cls_log
from src.seqlabel_utils import extract_entity, get_entity_span


import transformers 
transformers.logging.set_verbosity_error()
from transformers import BertTokenizer,AdamW, get_linear_schedule_with_warmup
device = get_torch_device()
set_seed()

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [4]:
tp = TrainParams(
    log_steps = 10,
    save_steps = 1000,
    epoch_size=20,
    max_seq_len=512,
    batch_size=20,
    loss_fn=nn.CrossEntropyLoss(),
    lr=5e-5,
    weight_decay=0.0,
    epsilon=1e-6,
    warmup_steps=100,
    dropout_rate=0.2,
    gradient_clip=5.0,
    hidden_size=200,
    early_stop_params = {
        'monitor':'f1_micro',
        'mode':'max',
        'min_delta': 0,
        'patience':3,
        'verbose':False
    },
    pretrain_model = 'bert-base-chinese',
    continue_train=False,
    label2idx ={'FIN':1},
    idx2label = {1:'FIN'},
    label_size=2,
    schema='span'
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(tp.pretrain_model, do_lower_case=True)
train_dataset = SpanDataset(data_loader('/kaggle/input/finent/train_span.txt'), tokenizer, tp.max_seq_len)
valid_dataset = SpanDataset(data_loader('/kaggle/input/finent/valid_span.txt'), tokenizer, tp.max_seq_len)
train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)

In [ ]:
tp.update({'num_train_steps': int(len(train_loader)*tp.epoch_size)})

CKPT = './checkpoint/span_tag'
saver = ModelSave(CKPT, continue_train=False)
saver.init()
es = EarlyStop(**tp.early_stop_params)
global_step = 0
tb = SummaryWriter(CKPT)

model = BertSpan(tp)
model.to(device)
optimizer, scheduler = model.get_optimizer()

In [8]:
for epoch_i in range(tp['epoch_size']):
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
    print("-"*60)

    # Measure the elapsed time of each epoch
    t0_epoch, t0_batch = time.time(), time.time()
    total_loss, batch_loss, batch_counts = 0, 0, 0

    model.train()
    for step, batch in enumerate(train_loader):
        global_step +=1
        batch_counts +=1

        #Forward propogate
        model.zero_grad()
        feature = {k:v.to(device) for k, v in batch.items()}
        logits = model(feature)
        loss = model.compute_loss(feature, logits)
        batch_loss += loss.item()
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), tp.gradient_clip)
        optimizer.step()
        scheduler.step()
        # Log steps for train loss logging
        if (step % tp.log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
            time_elapsed = time.time() - t0_batch
            print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
            tb.add_scalar('loss/batch_train', batch_loss / batch_counts, global_step=global_step)
            batch_loss, batch_counts = 0, 0
            t0_batch = time.time()

        # Save steps for ckpt saving and dev evaluation
        if (step % tp.save_steps == 0 and step != 0) or (step == len(train_loader) - 1):
            val_metrics = seq_span_metrics(model, valid_loader, tp.idx2label, device)
            for key, val in val_metrics.items():
                tb.add_scalar(f'metric/{key}', val, global_step=global_step)
            avg_train_loss = total_loss / step
            tb.add_scalars('loss/train_valid',{'train': avg_train_loss,
                                                'valid': val_metrics['val_loss']}, global_step=global_step)
            saver(total_loss / step, val_metrics['val_loss'], epoch_i, global_step, model, optimizer, scheduler)

    # On Epoch End: calcualte train & valid loss and log overall metrics
    time_elapsed = time.time() - t0_epoch
    val_metrics = seq_span_metrics(model, valid_loader, tp.idx2label, device)
    avg_train_loss = total_loss / step

    print("-"*70)
    print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
    tag_cls_log(epoch_i, val_metrics)
    print("\n")
    if es.check(val_metrics):
        break 

 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   10    |   1.267005   |     -     |   12.54  
   1    |   20    |   0.370860   |     -     |   10.56  
   1    |   30    |   0.124825   |     -     |   10.52  
   1    |   40    |   0.118028   |     -     |   10.44  
   1    |   50    |   0.102047   |     -     |   10.36  
   1    |   60    |   0.058101   |     -     |   10.36  
   1    |   70    |   0.038279   |     -     |   10.37  
   1    |   80    |   0.035708   |     -     |   10.30  
   1    |   90    |   0.028355   |     -     |   10.38  
   1    |   100   |   0.031106   |     -     |   10.31  
   1    |   110   |   0.028496   |     -     |   10.47  
   1    |   120   |   0.029313   |     -     |   10.38  
   1    |   130   |   0.029044   |     -     |   10.39  
   1    |   140   |   0.031391   |     -     |   10.36  
   1    |   150   |   0.027777   |     -     |   10.40  
   1    |   160   |   0.0

In [12]:
valid = pd.read_csv('./trainsample/valid.csv')
train = pd.read_csv('./trainsample/train.csv')
train.fillna({'entities':''},inplace=True)
valid.fillna({'entities':''},inplace=True)
known_entity = set(chain(*train['entities'].map(lambda x: x.split(';')).values))

In [40]:
pred = span_inference(model, valid_loader, device)
valid['pred_pos'] = [get_entity_span(i, tp.idx2label) for i in pred]
valid['pred_entity'] = valid.apply(lambda x: extract_entity(x.corpus, x.pred_pos)['FIN'], axis=1)
valid['pred_entity'] = valid['pred_entity'].map(lambda x: [i for i in x if islegal(i)])

In [41]:
stat = aggregate_f1(valid['id'].values, 
                  valid['entities'].values,
                  valid['pred_entity'].values, 
                  known_entity)
stat.to_csv('bert_span.csv')

All Entity Evaluation
   precision    recall        f1
0   0.628162  0.642241  0.635124
Unknown Entity Evalutation
   precision    recall        f1
0   0.440208  0.488462  0.463081
